'''
OBJECTIVES:
1. Build WRS system
2. Build Structural BMP Solution evaluator
3. Identify minimum BMP solution front for:
   individual facilities
   facilities w/in departments
   facilities w/in city
'''

In [1]:
'''
Define basic SQLAlchemy items:
    declarative base object
    connection object
    session object
    DB tables
'''
#SQLAlchemy library items:
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String
from sqlalchemy import update, insert
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship #http://docs.sqlalchemy.org/en/latest/orm/basic_relationships.html#relationship-patterns
from sqlalchemy import inspect

#deprecate working w/ dbs located on disk for now.
# dbfilename = '_jonhonda_files//test.db' #right now code writes to memory, later change to write to disk not used right now.
# print ("\nClearing old DB")
# try:
#     os.remove(dbfilename)
# except FileNotFoundError as err:
#     print ("no need to remove db file")####it's okay if file doesn't exist. ####
# engine = create_engine('sqlite:///'+ dbfilename, echo = False)

from SQLA_Base import Base #module containing declarative_base
from SQLA_conn_man import session, engine #module handling db and connection creation 

#NOW DEFINE DB SCHEMA (THIS DOESN'T WRITE SCHEMA TO DB, JUST TO SQLALCHEMY CLASSES AND OBJECTS)
#define an SQLAlchemy base class to maintain catalog of classes and tables relative to this base
from SQLA_DB_expressions import Expressions
from SQLA_DB_facility_chars import Facility_Chars
from SQLA_DB_facility_monthly_rain import Facility_Monthly_Rain
from SQLA_DB_facility_risks import Facility_Risks
from SQLA_DB_facility_types import Facility_Types
from SQLA_DB_wrs_pollutant_risks import WRS_Pollutant_Risks
Base.metadata.create_all(engine, checkfirst=True) #create tables

import SQLA_main as SQLA_main #import main SQLAlchemy functions


In [13]:
from CSVImporter import importCSV
'''
Dictionary of "SQLAlchemy where clause lambda functions" that tests record uniqueness.
e.g. entries in the 'name' field of the people table must be unique.
Therefore, we determine whether to insert/update a csv row into the people table based on
whether the csv row's name field value exists in the people table.

if table has no record uniqueness requirement, then enter: TableName:False
'''
unqTests = {
    'people': lambda RowVal: Base.metadata.tables['people'].c['name'] == RowVal['name'],
    'locations': False
}

# importCSV('testlab\\2_table_input.csv', unqTests)
# #test some reords
# for u, a in session.query(People, Locations).filter(People.id==Locations.people_id):
#     print (u,a)
    
importFeasibilityQuestionsCSV()

Reading csv for import to Feasibility Questions

Reading csv record: Feas-1
Adding to variable dictionary: OFFSITE_SD_Exist


NameError: name 'setRecord' is not defined

In [12]:
def importFeasibilityQuestionsCSV():
    import csv  
    #define csv column constants:
    C_fid_element = 0
    C_english_element = 1
    C_FacAvailableValVar = 2
    C_BMPReqdValVar = 4
    C_expression_element = 5
    print ('Reading csv for import to Feasibility Questions')
    with open('Input_Files\\feasibility_test_questions.csv', 'rt', encoding='utf-8-sig') as csvfile:
        csvreader = csv.reader(csvfile,dialect='excel')
        # expect and handle 1st row as header row
        rowi = 0
        for row in csvreader:
            if rowi > 0: 
                #read expression definition & write expression to expressions table
                Fid = row[C_fid_element]
                print ('\nReading csv record: ' + Fid)
                Question_english = row[C_english_element]
                Question_expression = row[C_expression_element]
                VarDict = {} #init empty VarDict for expression
    #             build variable def. for db_val types, assume val obtained from fac. char table in db_val field, w/ fac_id as unique queryable field            
    #             eval type based on type given. expect passed value to be val_type.value
                retStatus = procInputVarDecs(VarDict, row[C_FacAvailableValVar], 'facility_chars', 'USEDECVAL', 'facility_id', 'FLOAT')
                if isFuncStatusOK(retStatus[0]) == 1: # check if return status ok
                    retStatus= procInputVarDecs(VarDict, row[C_BMPReqdValVar], 'facility_chars', 'USEDECVAL', 'facility_id', 'FLOAT')
                    VarDict = retStatus[1]
                if isFuncStatusOK(retStatus[0]) == 0: #check if return status ok
                    print (getFuncStatusFault(retStatus[0]) + '    Fix error and retry.')
                else:
                    VarDict = retStatus[1]
        #          now write expression information:
                    expression_id = registerExpr (Fid, Question_expression, VarDict)
                    #now write feasibility_test_question record:
                    RecordFieldLS = ['feas_id', 'question_english', 'question_expression_id']
                    RecordValsTPL = (Fid,Question_english,expression_id)
                    print (RecordFieldLS)
    #                 feasibility_test_question_id = setRecord(dbFileName, 'feasibility_test_questions', RecordFieldLS, RecordValsTPL, 'feasibility_test_question_id','feas_id=(?)',(Fid,))
            rowi +=1

In [4]:
# functions to help with determining function status
# uses funcStatus list:
#func status is list showing func status: [0] = 0 or 1 (0 if fault, 1 if ok)
#                                         [1] = fault description
def isFuncStatusOK (funcStatus):
    #return 0 if fault, 1 if no fault
    return funcStatus[0]

def getFuncStatusFault (funcStatus):
    return funcStatus[1]

In [11]:
#functions to assist w/ expression and expression-variable declaration

def procInputVarDecs (VarDict, VarDecStr, WriteToTable, WriteToField, QryOnUniqueField, DataType):
#function to process variable declarations;
# return tuple: (funcStatus, VarDict)
#func status is list showing func status: [0] = 0 or 1 (0 if fault, 1 if ok)
#                                         [1] = fault description
# VarDict is the dictionary object that we will write to and return
# use the project standard VarDict: 
#    key = var name: [VarName, VarType, StoredTable, StoredField, QryOnUniqueField, DataType]
# VarDecStr is the string holding the csv variable declaration
#   expected format: var_type DELIMITER val
#           var_type: val; exp; dxp; val
#           DELIMITER: |
#           DECval:      var name; expr name; dynamic expr format:
#                dynamic_expr_format: dyn expr name(exprID_tablename~exprID_fieldname~unqFieldName)
#                identifies what table and field name holds reference to the expression_id, and the unique field  of the table that identifies the record. 


# WriteToTable & WriteToField are the table and field db_var type variable can be found at
# QryOnUniqueField is field that we should query on when trying to retrieve val from db table and db field
    funcStatus = [1,''] #start with okay status
    DELIMITER = '|'
    mySplitter = VarDecStr.split(DELIMITER) #split declaration into its 2 parts
    if len(mySplitter) != 2:
        if mySplitter[0] == 'N/A':
            return (funcStatus,VarDict)
        funcStatus = [0,'!!!!FAULT in procInputVarDecs: VarDecStr missing 2 parts: ' + str(VarDecStr)]
        return (funcStatus,VarDict)
    if WriteToField == 'USEDECVAL': #use declaration value as name for writing field
        WriteToField = mySplitter[1]
    if mySplitter[0] == 'var': #then it's value is held in the database
        VarDict =  add2VarDict(VarDict, mySplitter[1], 'val', WriteToTable,WriteToField, QryOnUniqueField, DataType)           
    elif mySplitter[0] == 'exp': #then it's value is defined by a static database held expression
        VarDict =  add2VarDict(VarDict, mySplitter[1], 'exp', 'expressions','expression_str', 'expression_id', DataType)
    elif mySplitter[0] == 'dxp': #then it's value is defined by a dynamically defined expression
        FullVarName = mySplitter[1] #'dyn expr name(exprID_tablename~exprID_fieldname~unqFieldName)'
        ParamPart = FullVarName[FullVarName.find('('):] #slice string to just the Parameter part, including open and close parens, 
        ParamPart = ParamPart[1:len(ParamPart)-1] #revise ParamPart to exclude open and close parens
        ParamSplitter = ParamPart.split('~')
        dyn_expr_name = initstr[:len(initstr)-len(ParamPart)]
        print ('my dxp: ' + dyn_expr_name)
        VarDict =  add2VarDict(VarDict, FullVarName, 'dxp', ParamSplitter[0],ParamSplitter[1], ParamSplitter[2], DataType)        
    elif mySplitter[0] != 'val': #the only other possible dec type is val. fault if not val
        funcStatus = [0,'!!!!FAULT in procInputVarDecs: Unknown var_type: ' + str(mySplitter[0])]
        return (funcStatus,VarDict)
    return (funcStatus,VarDict)

def add2VarDict (VarDict, VarName, VarType, StoredTable, StoredField, QryOnUniqueField, DataType):
# function to add variable definition to the given dictionary of variables
# use the project standard VarDict: 
#    key = var name: [VarName, VarType, StoredTable, StoredField, QryOnUniqueField, DataType]
    print('Adding to variable dictionary: ' + VarName)
    defList = [VarName, VarType, StoredTable, StoredField, QryOnUniqueField, DataType] #variable definition list
    key = VarName
    VarDict.update({key: defList})
    return VarDict

def registerExpr (expression_name, expression_str, VarDict):
#function to register expressions in db. 
#provide expression Str and var dict object
# assume any expressions in bar dict are already registered.
#Return expression id 
#determine if expression is in table
    import os
    import pickle
#     print ('Registering expression: ' + expression_name + '=' + expression_str)
# Pickle the 'data' dictionary using the highest protocol available.
# use the dumps command to write pickle to a string
    pickled_VarDict = pickle.dumps(VarDict, pickle.HIGHEST_PROTOCOL) ####uses the latest pickling version
#insert/update expression record, depending on if vars has data or not (if not, then it's assumed to be a const. val):    
    if len(VarDict) >0:
        RecordFieldLS = ['expression_name', 'expression_str', 'expression_data_type', 'vars']
        RecordValsTPL = (expression_name,expression_str,'FLOAT',pickled_VarDict)
    else:
        RecordFieldLS = ['expression_name', 'expression_str', 'expression_data_type']
        RecordValsTPL = (expression_name,expression_str,'FLOAT')        
def setRecord(dbConnection, dbTableName, RecordFieldLS, RecordValsTPL, PrimaryKeyField, WhereClause, WhereValTPL,flgCommitAfterTransaction=True):
def insertupdateRec(myTable, setFieldVals, whereConstraint):
    expression_id = SQLA_main.insertupdateRec(Expressions, )
    expression_id = setRecord(dbFileName, 'expressions', RecordFieldLS, RecordValsTPL, 'expression_id', 'expression_name = (?)', (expression_name,))
    return expression_id          

In [ ]:
session.close()
engine.dispose()